In [1]:
import numpy as np
import pandas as pd
import keras
from sklearn import preprocessing
from sklearn import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, RNN, LSTM, GRU, SpatialDropout1D, Dropout

In [2]:
import matplotlib.pyplot as plt
import os, os.path, sys
import copy

# TESTIRANJE ARHITEKTURE

In [5]:
path = r"C:\Users\mata2\Desktop\master\podaci\20k\Y"

In [6]:
dir = os.listdir(path)

Napravljen je pandas DataFrame gde su atributi zasebno pokrenute putanje, dok ce ciljna promenljiva da bude srednja vrednost svih putanja

In [7]:
file_names = []
for file in dir:
    file_names.append(file)

In [10]:
def create_timeseries_split(data, time_steps):

  # Get the number of samples in the data
  num_samples = len(data) - time_steps

  # Create empty arrays to store features and target values
  x_train = np.zeros((num_samples, time_steps, 1))
  y_train = np.zeros((num_samples, 1))

  # Loop through the data and create features and target values
  for i in range(num_samples):
    # Extract a slice of data for the current feature
    x_train[i] = data[i:i+time_steps, :]

    # The target value is the next value after the feature
    y_train[i] = data[i+time_steps, 0]

  return x_train, y_train

In [12]:
min_kord = [float('inf') for x in range(61000)]
max_kord = [float('-inf') for x in range(61000)]

for file in file_names:
  current_file = pd.read_csv(path+'/'+file, header=None)[:61000].astype('int')

  for i in range(len(current_file)):
    if current_file[0][i] < min_kord[i]:
      min_kord[i] = current_file[0][i]
    elif current_file[0][i] > max_kord[i]:
      max_kord[i] = current_file[0][i]

In [13]:
synth_path = r"C:\Users\mata2\Desktop\master\Axis-projection-RNN\sinteticki podaci\20k\Y"

In [14]:
dir2 = os.listdir(synth_path)

In [15]:
file_names_synt = []
i = 0
for file in dir2:
    i += 1
    if i > 100:
        break       
    else:
        file_names_synt.append(file)

In [17]:
train_synt = np.concatenate([pd.read_csv(synth_path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names_synt], axis=0)
test_synt = np.concatenate([pd.read_csv(path + "/" + file, header=None)[:61000].astype('int')
                              for file in file_names], axis=0)

In [16]:
time_steps = 1
input_shape = (time_steps,1)

In [22]:
train_X, train_y = create_timeseries_split(train_synt, time_steps)
test_X, test_y = create_timeseries_split(test_synt, time_steps)

In [23]:
gru_unit = [32,64]
unit1 = [32,64]
unit2 = [32,64]


batch_size = [1024,2048,4096]

In [24]:
best_skor = float('-inf')
best_gru = None
best_unit1 = None
best_unit2 = None
best_batch = None
best_LR = None

for gru in gru_unit:
    for uni1 in unit1:
        for uni2 in unit2:
            for batch in batch_size:

                keras.utils.set_random_seed(7)
                optimizerMSE = keras.optimizers.Adam(learning_rate=0.0001)

                model = Sequential()
                model.add(GRU(units = gru, input_shape = input_shape, activation = keras.layers.LeakyReLU()))
                model.add(Dense(units = uni1, activation=keras.layers.LeakyReLU()))
                model.add(Dense(units = uni2, activation=keras.layers.LeakyReLU()))
                model.add(Dense(units = 1, activation=keras.layers.LeakyReLU(), bias_initializer='zeros', kernel_initializer='normal'))
                model.compile(loss='mean_squared_error', optimizer = optimizerMSE)

                model.fit(train_X, train_y, epochs = 4, batch_size = batch, verbose = 2)
                predict = model.predict(test_X)
                
                train_scores = 0
                
                for i in range(61000):
                    if (min_kord[i] <= predict[i][0] <= max_kord[i]):
                        train_scores +=1

                if train_scores > best_skor:
                    best_skor = train_scores
                    best_gru = gru
                    best_unit1 = uni1
                    best_unit2 = uni2
                    best_batch = batch

                    print('best_skor: ', best_skor,'best_gru: ', best_gru,'best_unit1: ', best_unit1,'best_unit2: ', best_unit2,'best batch size: ', best_batch)
print('best_skor: ', best_skor)                    
print('best_gru: ', best_gru)
print('best_unit1: ', best_unit1)
print('best_unit2: ', best_unit2)
#print('best learning rate: ', best_LR)    
print('best batch size: ', best_batch)    


Epoch 1/4

5958/5958 - 18s - loss: 567804608.0000 - 18s/epoch - 3ms/step
Epoch 2/4
5958/5958 - 18s - loss: 1346.2710 - 18s/epoch - 3ms/step
Epoch 3/4
5958/5958 - 15s - loss: 1421.2921 - 15s/epoch - 3ms/step
Epoch 4/4
5958/5958 - 15s - loss: 1470.5815 - 15s/epoch - 3ms/step
57188/57188 [==============================] - 55s 962us/step
best_skor:  19046 best_gru:  32 best_unit1:  32 best_unit2:  32 best batch size:  1024
Epoch 1/4
2979/2979 - 13s - loss: 1130943872.0000 - 13s/epoch - 4ms/step
Epoch 2/4
2979/2979 - 12s - loss: 1343.0488 - 12s/epoch - 4ms/step
Epoch 3/4
2979/2979 - 12s - loss: 1343.3427 - 12s/epoch - 4ms/step
Epoch 4/4
2979/2979 - 12s - loss: 1344.8489 - 12s/epoch - 4ms/step
57188/57188 [==============================] - 59s 1ms/step
best_skor:  46505 best_gru:  32 best_unit1:  32 best_unit2:  32 best batch size:  2048
Epoch 1/4
1490/1490 - 12s - loss: 2256135936.0000 - 12s/epoch - 8ms/step
Epoch 2/4
1490/1490 - 9s - loss: 1342.9208 - 9s/epoch - 6ms/step
Epoch 3/4
1490/149

57188/57188 [==============================] - 58s 1ms/step
Epoch 1/4
1490/1490 - 19s - loss: 1081037440.0000 - 19s/epoch - 13ms/step
Epoch 2/4
1490/1490 - 17s - loss: 1342.9189 - 17s/epoch - 11ms/step
Epoch 3/4
1490/1490 - 17s - loss: 1342.9471 - 17s/epoch - 11ms/step
Epoch 4/4
1490/1490 - 17s - loss: 1343.0142 - 17s/epoch - 11ms/step
57188/57188 [==============================] - 57s 992us/step
best_skor:  56203
best_gru:  64
best_unit1:  32
best_unit2:  32
best batch size:  2048
